# Running a Molecular Dynamics Simulation with Hoomd-Polymers

## Overview:
In this tutorial, we will run a molecular dynamics simulation of Polyphenylene sulfide molecules using the Hoomd-Polymers package. We will use the [`HOOMD-blue`](https://hoomd-blue.readthedocs.io/en/v4.1.0/) simulation engine to run the simulation and the `hoomd_organics` package to initialize the system of polymer configuration.

In summary, the `hoomd-polymers` package has three main classes:

-  `Molecule`: This class is used to define the structure of a molecule (for example the structure of a polymer built from a monomer).

- `System`: This class is used to define the system of molecules (for example a system of polymers) in a box and creates the initial `gsd` snapshot of the system. It also applies the forcefiled to the system and prepares the required forces for the simulation.

- `Simulation`: This class is used to run the simulation using the `HOOMD-blue` simulation engine. In order to initialize a simulation, a `gsd` snapshot of the system and a list of `Hoomd` forces are required.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from hoomd_organics.library import PolyEthylene, OPLS_AA
from hoomd_organics import Pack
from hoomd_organics.modules.welding import SlabSimulation, Interface, WeldSimulation

import gsd
import pickle
import unyt as u

### Step 1: Creating a "Slab"



In [ ]:
molecule = PolyEthylene(num_mols=30, lengths=10)

system = Pack(
    molecules=molecule,
    force_field=OPLS_AA(),
    density=1.1,
    r_cut=2.5,
    auto_scale=True,
    remove_charges=True,
    remove_hydrogens=True
)

In [ ]:
sim = SlabSimulation.from_system(
    system=system,
    interface_axis=(1,0,0),
    gsd_file_name="slab_x_interface.gsd",
)

In [ ]:
sim.forces

In [ ]:
for wall in sim.forces[-1].walls:
    print(wall)
print()
print("Simulation volume:", sim.box_lengths_reduced)

In [ ]:
sim.run_update_volume(final_density=1.2*u.g/(u.cm**3), n_steps=5e4, kT=5.0, period=100, tau_kt=0.001)

In [ ]:
sim.density.to(u.g/(u.cm**3))

In [ ]:
sim.run_NVT(kT=5.0, n_steps=5e4, tau_kt=0.001)

In [ ]:
sim.pickle_forcefield()

In [ ]:
# Visualize slab, wrapped and unwrapped

In [ ]:
sim.forces

### Step 2: Creating an interface from the slab



In [ ]:
interface = Interface(gsd_file="slab_x_interface.gsd", interface_axis=(1, 0, 0), gap=0.1)

In [ ]:
print(type(interface.hoomd_snapshot))
print("Slab number of particles:", system.n_particles)
print("Interface number of particles:", interface.hoomd_snapshot.particles.N)
print()
print("Slab box lengths:", sim.box_lengths_reduced)
print("Interface box lengths:", interface.hoomd_snapshot.configuration.box[:3])

### Step 3: Running a welding simulation



In [ ]:
weld_sim = WeldSimulation(initial_state=interface.hoomd_snapshot, forcefield=sim.forces[:5])

In [ ]:
weld_sim.forces

In [ ]:
for wall in weld_sim.forces[-1].walls:
    print(wall)
print()
print("Simulation volume:", weld_sim.box_lengths_reduced)

In [ ]:
weld_sim.run_NVT(kT=6.0, n_steps=5e4, tau_kt=0.001)